# MOFs LM Probing for Linker

In [1]:
from baseline import PromptSet

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [2]:
file = "../data/mof_linkers.csv"

In [3]:
df = pd.read_csv(file)
df.head()

,SubjectEntity,Relation,ObjectEntity
0,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-]
1,MOF-CJ3,hasOrganicLinkers,[O-]C(=O)c1cc(cc(c1)C(=O)[O-])C(=O)[O-]
2,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)C1=NC(=[N]=C([N]1)c1ccc(cc1...
3,NJU-Bai9,hasOrganicLinkers,O=C(c1cncc(c1)C(=O)Nc1ccc(cc1)C(=O)[O-])Nc1ccc...
4,MOF-1,hasOrganicLinkers,[O-]C(=O)c1cccc(c1)OCc1c(COc2cccc(c2)C(=O)[O-]...


In [4]:
df.shape

(1735, 3)

In [5]:
df.ObjectEntity.nunique()

538

## Probing BERT

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [7]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [8]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [9]:
def create_linker_prompt1(sub, mask):
    return f"{sub} contains a {mask} which is organic linker."
    
def create_linker_prompt2(sub, mask):
    return f"{sub} contains an organic linker which is {mask}."

def create_linker_prompt3(sub, mask):
    return f"The metal clusters in {sub} are connected by {mask}."

def create_linker_prompt4(sub, mask):
    return f"{sub} is an MOF made of metal centers connectd by organic linker {mask}."
    
def create_linker_prompt5(sub, mask):
    return f"{sub} is an MOF material containing metal centers and organic linkers {mask}."
    
def create_linker_prompt6(sub, mask):
    return f"{sub} is an MOF structure with metal centers and organic linkers {mask}." 
    
def create_linker_prompt7(sub, mask):
    return f"The organic linker in MOF {sub} is {mask}."
    
def create_linker_prompt8(sub, mask):
    return f"{sub} has an organic linker with SMILES string {mask}."
    
def create_linker_prompt9(sub, mask):
    return f"{sub} has SBUs and organic linkers {mask}."
    
def create_linker_prompt10(sub, mask):
    return f"{sub} is a type of polymer with metal clusters and organic linkers {mask}."
    
def create_linker_prompt11(sub, mask):
    return f"{mask} is the structure of the organic linker in MOF {sub}."
    
def create_linker_prompt12(sub, mask):
    return f"The structure of the organic linker of MOF {sub} is {mask}."

In [10]:
processors = [create_linker_prompt1, create_linker_prompt2,
              create_linker_prompt3, create_linker_prompt4,
              create_linker_prompt5, create_linker_prompt6,
              create_linker_prompt7, create_linker_prompt8,
              create_linker_prompt9, create_linker_prompt10,
              create_linker_prompt11, create_linker_prompt12
             ]

## Test Small

In [46]:
processors = [create_linker_prompt1, create_linker_prompt2, create_linker_prompt3]

In [47]:
processors[0]('MOF-1', '[MASK]')

'MOF-1 contains a SMILES string [MASK] which is organic linker.'

In [48]:
mofs_sub =df[:10]

In [49]:
prompts = PromptSet([processors[0](row.SubjectEntity, mask_token) for _, row in mofs_sub.iterrows()])

In [50]:
prompts.__getitem__(9)

'SNU-50 contains a SMILES string [MASK] which is organic linker.'

In [51]:
for i in prompts:
    print(i)

MROF-1 contains a SMILES string [MASK] which is organic linker.
MOF-CJ3 contains a SMILES string [MASK] which is organic linker.
catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-triyltribenzoato)-dodeca-aqua-dodeca-copper dimethylsulfoxide solvate hydrate)|PCN-6|Teaching Subset: MOFs contains a SMILES string [MASK] which is organic linker.
NJU-Bai9 contains a SMILES string [MASK] which is organic linker.
MOF-1 contains a SMILES string [MASK] which is organic linker.
MOF-2 contains a SMILES string [MASK] which is organic linker.
TCM-9-Ho contains a SMILES string [MASK] which is organic linker.
catena-[(μ8-4,4'-(Hexafluoroisopropylidene)diphthalato)-aqua-di-cadmium(ii) hemihydrate] contains a SMILES string [MASK] which is organic linker.
catena-[bis(μ4-Hydrogen zoledronato)-bis(μ2-1,2-bis(4-pyridyl)ethene)-tri-nickel unknown solvate tetrahydrate] contains a SMILES string [MASK] which is organic linker.
SNU-50 contains a SMILES string [MASK] which is organic linker.


In [52]:
# Run the model
outputs = []
for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
    outputs.append(out)

  0%|          | 0/10 [00:00<?, ?it/s]

In [53]:
outputs

[[{'score': 0.7801147103309631,
   'token': 117,
   'token_str': ',',
   'sequence': 'MROF - 1 contains a SMILES string, which is organic linker.'},
  {'score': 0.11471892893314362,
   'token': 1200,
   'token_str': '##er',
   'sequence': 'MROF - 1 contains a SMILES stringer which is organic linker.'},
  {'score': 0.007421981077641249,
   'token': 13196,
   'token_str': 'module',
   'sequence': 'MROF - 1 contains a SMILES string module which is organic linker.'},
  {'score': 0.007102462463080883,
   'token': 14622,
   'token_str': 'generator',
   'sequence': 'MROF - 1 contains a SMILES string generator which is organic linker.'},
  {'score': 0.0067765191197395325,
   'token': 10926,
   'token_str': 'stack',
   'sequence': 'MROF - 1 contains a SMILES string stack which is organic linker.'},
  {'score': 0.003444866044446826,
   'token': 4442,
   'token_str': 'device',
   'sequence': 'MROF - 1 contains a SMILES string device which is organic linker.'},
  {'score': 0.0029897592030465603,
 

In [26]:
results = []
for idx, row in mofs_sub.iterrows():
    for output in outputs[idx]:
        record = {}
        record['SubjectEntity'] = row['SubjectEntity']
        record['Relation'] = row['Relation']
        record['ObjectEntity'] = row['ObjectEntity']
        record['prompt']= prompts[idx]
        record['predictedScore'] = output['score']
        record['predictedToken'] = output['token_str']
        results.append(record)

In [27]:
predicted_df = pd.DataFrame(results)
predicted_df

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken
0,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.071169,","
1,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.037063,oxygen
2,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.022421,a
3,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.021445,catalyst
4,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.019612,:
...,...,...,...,...,...,...
995,SNU-50,hasOrganicLinkers,O=C1[C][C]C(=O)N1c1cc(cc(c1)C(=O)[O-])C(=O)[O-],SNU-50 contains [MASK] which is organic linker.,0.001394,channel
996,SNU-50,hasOrganicLinkers,O=C1[C][C]C(=O)N1c1cc(cc(c1)C(=O)[O-])C(=O)[O-],SNU-50 contains [MASK] which is organic linker.,0.001389,pin
997,SNU-50,hasOrganicLinkers,O=C1[C][C]C(=O)N1c1cc(cc(c1)C(=O)[O-])C(=O)[O-],SNU-50 contains [MASK] which is organic linker.,0.001369,in
998,SNU-50,hasOrganicLinkers,O=C1[C][C]C(=O)N1c1cc(cc(c1)C(=O)[O-])C(=O)[O-],SNU-50 contains [MASK] which is organic linker.,0.001358,residue


### Evaluate on Small

In [28]:
predicted_df.loc[10]['ObjectEntity'].split(",")

['[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-]']

In [30]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        if token.lower() == objectEntities.lower():
            return 1
       
    return 0

In [31]:
# test what the function does
def hasPredicted_demo(objectEntities, token):
    #print(objectEntities)
    if pd.notna(objectEntities):
        if token.lower() == objectEntities.lower():
            return 1
       
    return 0

In [32]:
hasPredicted_demo('[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-]', 'Framework')

0

In [33]:
predicted_df['predicted'] = predicted_df.apply(hasPredicted, axis=1)

In [34]:
predicted_df.head()

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken,predicted
0,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.071169,",",0
1,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.037063,oxygen,0
2,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.022421,a,0
3,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.021445,catalyst,0
4,MROF-1,hasOrganicLinkers,[O-]C(=O)c1ccc(cc1)c1ccc(cc1)C(=O)[O-],MROF-1 contains [MASK] which is organic linker.,0.019612,:,0


In [35]:
predicted_df['predicted'].sum()

0

In [38]:
predicted_df[predicted_df.predicted == 1]

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken,predicted


In [39]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [40]:
topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=100)

In [41]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [42]:
topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')

In [43]:
topk_agg_df.columns = ['SubjectEntity', 'predicted']
topk_agg_df

,SubjectEntity,predicted
0,MOF-1,0
1,MOF-2,0
2,MOF-CJ3,0
3,MROF-1,0
4,NJU-Bai9,0
5,SNU-50,0
6,TCM-9-Ho,0
7,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",0
8,"catena-[(μ8-4,4'-(Hexafluoroisopropylidene)dip...",0
9,"catena-[bis(μ4-Hydrogen zoledronato)-bis(μ2-1,...",0


In [44]:
topk_agg_df.predicted.mean()

0.0

## Test BERT

In [11]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [12]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        if token.lower() == objectEntities.lower():
            return 1
       
    return 0

In [13]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [14]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [15]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

...

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

CPU times: user 3h 34min 12s, sys: 1min 10s, total: 3h 35min 23s
Wall time: 16min 59s


In [18]:
hasType_df = pd.DataFrame(acc_list)

In [19]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
56,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,10
26,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,10
2,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,10
69,hasMetals,The structure of the organic linker of MOF {SubjectEntity} is [MASK].,0.0,20
65,hasMetals,[MASK] is the structure of the organic linker in MOF {SubjectEntity}.,0.0,100
22,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,50
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,1
37,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,5
42,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,1
16,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,50


In [20]:
result_df.to_csv("../data/predicted_hasLinker_BERT.csv", index=None)

In [21]:
hasType_df.to_csv("../data/predicted_hasLinker_BERT_eval.csv", index=None)

In [22]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
6,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,1
12,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an MOF structure with metal centers and organic linkers [MASK].,0.0,1
36,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,1
42,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,1


In [23]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
1,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,100
22,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,50
21,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,20
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10


## Probing MatBERT

In [24]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/matbert-base-cased")

In [25]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [26]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [27]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

...

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

CPU times: user 1h 15min 46s, sys: 24.6 s, total: 1h 16min 10s
Wall time: 7min 2s


In [28]:
hasType_df = pd.DataFrame(acc_list)

In [29]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
5,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,100
11,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,100
4,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,50
15,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,20
53,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,100
47,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,100
26,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,10
8,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,10
68,hasMetals,The structure of the organic linker of MOF {SubjectEntity} is [MASK].,0.0,10
46,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,50


In [30]:
result_df.to_csv("../data/predicted_hasLinker_MatBERT.csv", index=None)

In [31]:
hasType_df.to_csv("../data/predicted_hasLinker_MatBERT_eval.csv", index=None)

In [32]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
6,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,1
12,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an MOF structure with metal centers and organic linkers [MASK].,0.0,1
36,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,1
42,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,1


In [33]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
1,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,100
22,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,50
21,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,20
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10


## Probing MatSciBERT

In [34]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [35]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [36]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [37]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

...

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

CPU times: user 1h 17min 32s, sys: 2.5 s, total: 1h 17min 35s
Wall time: 7min 8s


In [38]:
hasType_df = pd.DataFrame(acc_list)

In [39]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
65,hasMetals,[MASK] is the structure of the organic linker in MOF {SubjectEntity}.,0.0,100
7,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,5
41,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,100
17,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,100
71,hasMetals,The structure of the organic linker of MOF {SubjectEntity} is [MASK].,0.0,100
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10
47,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,100
38,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,10
5,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,100


In [40]:
result_df.to_csv("../data/predicted_hasLinker_MatSciBERT.csv", index=None)

In [41]:
hasType_df.to_csv("../data/predicted_hasLinker_MatSciBERT_eval.csv", index=None)

In [42]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
6,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,1
12,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an MOF structure with metal centers and organic linkers [MASK].,0.0,1
36,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,1
42,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,1


In [43]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
1,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,100
22,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,50
21,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,20
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10


## Probing SciBERT

In [44]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [45]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [46]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [47]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

...

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

CPU times: user 1h 18min 4s, sys: 2.38 s, total: 1h 18min 6s
Wall time: 7min 10s


In [48]:
hasType_df = pd.DataFrame(acc_list)

In [49]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
57,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,20
9,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,20
47,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,100
2,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,10
60,hasMetals,[MASK] is the structure of the organic linker in MOF {SubjectEntity}.,0.0,1
46,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,50
59,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,100
55,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,5
25,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,5
49,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,5


In [50]:
result_df.to_csv("../data/predicted_hasLinker_SciBERT.csv", index=None)

In [51]:
hasType_df.to_csv("../data/predicted_hasLinker_SciBERT_eval.csv", index=None)

In [52]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
6,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,1
12,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an MOF structure with metal centers and organic linkers [MASK].,0.0,1
36,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,1
42,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,1


In [53]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
1,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,100
22,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,50
21,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,20
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10


## Probing RoBERTa

In [54]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [55]:
mask_token = tokenizer.mask_token
mask_token

'<mask>'

## Creat the Prompt Pipeline

In [56]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [57]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

...

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

  0%|          | 0/1735 [00:00<?, ?it/s]

CPU times: user 3h 34min 53s, sys: 4.49 s, total: 3h 34min 57s
Wall time: 16min 56s


In [58]:
hasType_df = pd.DataFrame(acc_list)

In [59]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
40,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,50
41,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,100
19,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,5
46,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,50
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10
55,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,5
25,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,5
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
1,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,5


In [60]:
result_df.to_csv("../data/predicted_hasLinker_RoBERTa.csv", index=None)

In [61]:
hasType_df.to_csv("../data/predicted_hasLinker_RoBERTa_eval.csv", index=None)

In [62]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
6,hasMetals,{SubjectEntity} contains an organic linker which is [MASK].,0.0,1
12,hasMetals,The metal clusters in {SubjectEntity} are connected by [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF material containing metal centers and organic linkers [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an MOF structure with metal centers and organic linkers [MASK].,0.0,1
36,hasMetals,The organic linker in MOF {SubjectEntity} is [MASK].,0.0,1
42,hasMetals,{SubjectEntity} has an organic linker with SMILES string [MASK].,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is a type of polymer with metal clusters and organic linkers [MASK].,0.0,1


In [63]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,1
1,hasMetals,{SubjectEntity} contains a [MASK] which is organic linker.,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,100
22,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,50
21,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,20
20,hasMetals,{SubjectEntity} is an MOF made of metal centers connectd by organic linker [MASK].,0.0,10
